In [11]:
import import_ipynb
from ElektronImFeld import ElektronImFeld
from ExampleSmartis import ExampleSmartis
from Leiterschleife import Leiterschleife
from Kondensator import Kondensator
from KomplexeWechselstromrechnung import KomplexeWechselstromrechnung
from Widerstandsnetzwerk import Widerstandsnetzwerk
from ASmartis import *   
from Buttons import *   
from Chart import *
from ipywidgets import HBox, VBox, Layout,Box
from math import pi, sin, cos
from IPython.display import display, clear_output
import json
#clear_output()

class Replay(ASmartis):
    
    def __init__(self,parameterList):
        
        super().__init__(parameterList)
        self.updateTime = 0.5
        
        super().stopSendInformation()
        self.smartisDict ={"ElektronImFeld":ElektronImFeld,"ExampleSmartisTest":ExampleSmartis, "KomplexeWechselstromrechnung":KomplexeWechselstromrechnung,
                           "Leiterschleife":Leiterschleife, "Kondensator":Kondensator, "Widerstandsnetzwerk":Widerstandsnetzwerk}
        self.smartisObjectDict = {}
        self.createSmartisObjects()
        self.getCurrentSmartis()
        self.replayIndex = 0
        self.dataList = []
        
    def createButtons(self):        
        
        #Erzeuge das Object, welches alle Buttons managt
        self.buttonsObject = Buttons(self)        
        
        #Erzeuge verschiedene Buttons        
        dropdown_smartis = self.buttonsObject.newDropdown("dd_smartis", [], "smartiS:")
        dropdown_sessionID = self.buttonsObject.newDropdown("dd_sessionID", [], "Session-ID:")
        button_replay = self.buttonsObject.newReset("b_reset", 30,30)  

     
        #Aktualisiere ein Dictionary mit den Stati der Buttons        
        self.buttonsStatesDict = self.buttonsObject.getStates()
        self.oldButtonsStatesDict = dict(self.buttonsStatesDict)
        
        
        self.initalizeSmartisWidget()
        self.initalizeSessionIDWidget()
            
    def createChart(self):
        pass

        
    def createSmartisObjects(self):        
        
        for name in self.smartisNames:            
            s = name.split("-")
            smartis = s[0]
            arg = s[1].replace("(", "").replace(")", "")
            try:
                arg = json.loads(arg)
            except:
                print((arg))
            newSmartis = self.smartisDict[smartis](arg)    
            newSmartis.stopSendInformation()
            newSmartis.updateTime = 0.01

            newButtonsObject = newSmartis.getButtonsObject()
            newButtonsObject.disable_all(True)
            
            self.smartisObjectDict[name] = newSmartis
            
            
    def visualize(self): 
              
        
        #Darstellung   
        buttonsDict = self.buttonsObject.getButtons()
        display(HBox([buttonsDict["dd_smartis"],buttonsDict["dd_sessionID"],buttonsDict["b_reset"]]))
       
        self.out = widgets.Output()
        display(self.out)
        
    
    def visualizeSmarti(self):
        
        with self.out:
            clear_output()
            self.currentSmartis.visualize()
            self.currentButtonsObject.disable_all(True)
    
    def getCurrentSmartis(self):
        
        self.currentSmartis = self.smartisObjectDict[self.buttonsStatesDict["dd_smartis"]]            
        self.currentButtonsObject = self.currentSmartis.getButtonsObject()
             
    def update(self):
        
        if self.oldButtonsStatesDict["dd_smartis"]!= self.buttonsStatesDict["dd_smartis"]:
            
            self.reset()
           
            self.getCurrentSmartis()
            self.visualizeSmarti()
            self.initalizeSessionIDWidget()
            
        if  self.buttonsStatesDict["b_reset"]:
            self.reset()
            
            if self.buttonsStatesDict["dd_sessionID"]!= None:
                self.dataList = self.server.getSession(self.buttonsStatesDict["dd_smartis"],self.buttonsStatesDict["dd_sessionID"])
                
                self.replayIndex = 0
                self.currentSmartis.start()

                self.startReplay()
                self.buttonsObject.changeState("b_reset",False)
            else:
                self.buttonsObject.changeState("b_reset",False)

                
                
    def initalizeSmartisWidget(self):
        
        self.smartisNames = self.server.getAllSmartisNames()
        self.buttonsObject.changeOptions("dd_smartis",self.smartisNames)
     
    
    def initalizeSessionIDWidget(self):
        
        sessionID = self.server.getSessionIDofSmartis(self.buttonsStatesDict["dd_smartis"])
        self.buttonsObject.changeOptions("dd_sessionID",sessionID)
        
        
    def startReplay(self):
        
        
        #ListPositions:
        sessionIDPosition = 0
        smartisPosition = 1
        smartisArgumentsPosition = 2
        timePosition = 3
        buttonPosition = 4
        valuePosition = 5
        dataLength = len(self.dataList)
        i = self.replayIndex
        currentData = self.dataList[i]
        
        
        
             
        self.currentButtonsObject.changeState(currentData[buttonPosition],self.castingValue(currentData[valuePosition]))
        print(currentData[buttonPosition],self.castingValue(currentData[valuePosition]))
        currentTime = datetime.strptime(currentData[timePosition], "%m/%d/%Y, %H:%M:%S.%f")-timedelta(seconds=0)
        self.currentButtonsObject.disable_all(True)
        
        if i < dataLength-1:
            nextTime = datetime.strptime(self.dataList[i+1][timePosition], "%m/%d/%Y, %H:%M:%S.%f")-timedelta(seconds=0)
            timeDiff = nextTime-currentTime+timedelta(seconds=self.updateTime)
            #print(timeDiff.total_seconds())    
            threading.Timer(round(timeDiff.total_seconds(),3), self.startReplay).start()
            self.replayIndex+=1
        
        
    def castingValue(self,value):
        
        if "[" in value:
            value = value.replace("[", "")
            value = value.replace("]", "")
            value = value.split(",")
            newValues = [float(i) for i in value]
            return newValues
        
        if value == "True":
            return bool(value)
        elif value == "False":
            return False
        else:
            try:
                a = float(value)
                b = int(value)
                if a != b:
                    return a
                else:
                    return b
            except:
                #raise ValueError("failed to coerce str to int or float")
                return value
    
    def reset(self):
        try:
            self.currentSmartis.stop()
            self.currentSmartis.reset()
        except:
            pass
        


            
    def clearFile(self):
        
        self.server.clearFile()

           
        
    
            

In [13]:
r = Replay(None)
r.visualize()
r.start()



None
None


Output()

In [3]:
r.clearFile()

NameError: name 'r' is not defined

In [4]:

#display(print("bluB"))
a =KomplexeWechselstromrechnung(None)
a.visualize()
a.start()